In [42]:
!pip install kaggle

configure the path for kaggle.json

In [43]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [44]:
!kaggle datasets download -d kazanova/sentiment140


Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


**extract the zip file**

In [45]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()

import the dependencies

In [66]:
import numpy as np
import pandas as pd
#Used for text preprocessing, such as removing unwanted characters, tokenizing text, or extracting specific patterns.
import re
#Provides a list of common stopwords (e.g., “and”, “the”, “is”) that can be removed from the text to reduce noise.
from nltk.corpus import stopwords
#Used for stemming, which reduces words to their root form (e.g., “running” to “run”).
from nltk.stem.porter import PorterStemmer
#Converts text data into numerical features using Term Frequency-Inverse Document Frequency (TF-IDF).
from sklearn.feature_extraction.text import TfidfVectorizer
#Splits a dataset into training and testing sets, allowing machine learning models to be evaluated on unseen data.
from sklearn.model_selection import train_test_split
#A machine learning model used for classification tasks.
from sklearn.linear_model import LogisticRegression
#Evaluates the performance of the model by calculating the accuracy.
from sklearn.metrics import accuracy_score


In [47]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [48]:
#printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

data proessing

In [49]:
#loading the data to a csv
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [50]:
#CHECK THE NO OF ROWS AND COLUMN
twitter_data.shape

(1599999, 6)

In [51]:
 #first five rows of the dataset
 twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [52]:
#naming the column and reading the dataset

column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', names=column_names)
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [53]:
#counting the no of missing value in the dataset
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [54]:
#checking the distribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


In [55]:
#convert the target 4 to 1
twitter_data['target']=twitter_data.target.replace(4,1)

In [56]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


 0 = negative tweet

 1 = positive tweet

**Steming**

which reduces words to their root form (e.g., “running” to “run”)

In [57]:
port_stem = PorterStemmer()

In [58]:
def steming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content


In [59]:
texts = twitter_data['text']
print(text)

0          @switchfoot http://twitpic.com/2y1zl - Awww, t...
1          is upset that he can't update his Facebook by ...
2          @Kenichan I dived many times for the ball. Man...
3            my whole body feels itchy and like its on fire 
4          @nationwideclass no, it's not behaving at all....
                                 ...                        
1599995    Just woke up. Having no school is the best fee...
1599996    TheWDB.com - Very cool to hear old Walt interv...
1599997    Are you ready for your MoJo Makeover? Ask me f...
1599998    Happy 38th Birthday to my boo of alll time!!! ...
1599999    happy #charitytuesday @theNSPCC @SparksCharity...
Name: text, Length: 1600000, dtype: object


In [60]:
steming(texts[0])

'switchfoot http twitpic com zl awww bummer shoulda got david carr third day'

In [62]:
twitter_data['stemmed_content']=texts.apply(steming)

In [63]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [64]:
x = twitter_data['stemmed_content']
y = twitter_data['target']

split the data into taining and testing dataset

In [68]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [69]:
print(x.shape,x_train.shape,x_test.shape)

(1600000,) (1280000,) (320000,)


In [70]:
print(x_train)

1570269                          watch saw iv drink lil wine
1273074                                         hatermagazin
88479      even though favourit drink think vodka coke wi...
254604                   think hand got burnt sun today hurt
667941     took mazi dr shot today come find ear infect p...
                                 ...                        
941805                                       threewink cheer
1007131    vote livewir play live smith tomorrow night su...
1460311                               eager monday afternoon
929226     hope everyon mother great day wait hear guy st...
526253                      love wake folger bad voic deeper
Name: stemmed_content, Length: 1280000, dtype: object


In [71]:
print(y_train)

1570269    1
1273074    1
88479      0
254604     0
667941     0
          ..
941805     1
1007131    1
1460311    1
929226     1
526253     0
Name: target, Length: 1280000, dtype: int64


convert the data into numerical features

In [72]:
vectorizer = TfidfVectorizer()
vectorizer.fit(x_train)

x_train = vectorizer.transform(x_train)
x_test = vectorizer.transform(x_test)

In [74]:
print(x_train)

  (0, 443066)	0.4484755317023172
  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 235045)	0.41996827700291095
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (1, 160636)	1.0
  (2, 443430)	0.3348599670252845
  (2, 433560)	0.3296595898028565
  (2, 409143)	0.15169282335109835
  (2, 407301)	0.18709338684973031
  (2, 406399)	0.32105459490875526
  (2, 288470)	0.16786949597862733
  (2, 266729)	0.24123230668976975
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 129411)	0.29074192727957143
  (2, 124484)	0.1892155960801415
  (2, 109306)	0.4591176413728317
  (2, 77929)	0.31284080750346344
  (3, 411528)	0.27089772444087873
  (3, 406399)	0.29029991238662284
  (3, 388626)	0.3940776331458846
  (3, 172421)	0.37464146922154384
  :	:
  (1279996, 373144)	0.35212500999832036
  (1279996, 318303)	0.21254698865277744
  (1279996, 291078)	0.17981734369155505
  (1279996, 238078)	0.5606696159563151
  (1279996, 2

In [75]:
print(x_test)

  (0, 420984)	0.17915624523539803
  (0, 409143)	0.31430470598079707
  (0, 398906)	0.3491043873264267
  (0, 388348)	0.21985076072061738
  (0, 279082)	0.1782518010910344
  (0, 271016)	0.4535662391658828
  (0, 171378)	0.2805816206356073
  (0, 138164)	0.23688292264071403
  (0, 132364)	0.25525488955578596
  (0, 106069)	0.3655545001090455
  (0, 67828)	0.26800375270827315
  (0, 31168)	0.16247724180521766
  (0, 15110)	0.1719352837797837
  (1, 366203)	0.24595562404108307
  (1, 348135)	0.4739279595416274
  (1, 256777)	0.28751585696559306
  (1, 217562)	0.40288153995289894
  (1, 145393)	0.575262969264869
  (1, 15110)	0.211037449588008
  (1, 6463)	0.30733520460524466
  (2, 400621)	0.4317732461913093
  (2, 256834)	0.2564939661498776
  (2, 183312)	0.5892069252021465
  (2, 89448)	0.36340369428387626
  (2, 34401)	0.37916255084357414
  :	:
  (319994, 123278)	0.4530341382559843
  (319995, 444934)	0.3211092817599261
  (319995, 420984)	0.22631428606830145
  (319995, 416257)	0.23816465111736276
  (319995, 3

traing the machine learning model

Logical Regression

In [78]:
regressor = LogisticRegression(max_iter=1000)

In [79]:
regressor.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

model evaluation

Accuracy score on training data

In [92]:
x_pred_train = regressor.predict(x_train)

In [96]:
x_train_accuracy = accuracy_score(y_train,x_pred_train)

In [97]:
print('Accuracy score on training data:',x_train_accuracy)

Accuracy score on training data: 0.8102125


In [91]:
#Accuracy score on testing data
x_pred_test = regressor.predict(x_test)
x_test_accuracy = accuracy_score(y_test,x_pred_test)

In [95]:
print('Accuracy score on test data:',x_test_accuracy)

Accuracy score on training data: 0.778021875


Model Accuracy

# Saving the trained model

In [98]:
import pickle

In [99]:
filename = 'trained_model.sav'
pickle.dump(regressor,open(filename,'wb'))

In [160]:
file = open('trained_model.txt','wb')
pickle.dump(regressor,file)

Using the saved model for future predictions

In [101]:
loaded_model = pickle.load(open('/content/trained_model.sav','rb'))

In [111]:
x_new = x_test[3]
prediction = loaded_model.predict(x_new)
print(prediction)
if (prediction[0]==0):
  print('Negative tweet')
else:
  print('Positive tweet')

[0]
Negative tweet


# Text Sentiment Analysis

to check the text whether it is positive or negative sentiment

In [117]:
text = str(input('ENTER THE STATEMENT TO VERIFY SENTIMENT'))
text = steming(text)
input_data = vectorizer.transform([text])
prediction = loaded_model.predict(input_data)
print(prediction)
if (prediction[0]==0):
  print('Negative tweet')
else:
  print('Positive tweet')

ENTER THE STATEMENT TO VERIFY SENTIMENTWHEN WILL YOU COME I WANT TO KILL YOU
[0]
Negative tweet


In [130]:
!pip install article
!pip install newspaper3k

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 39.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 7.4 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=3dca302bc7604c280b9a04d07ec9c05760c7b6cc37245dc345a7323c0f0ce2a0
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3342 sha256=e811ab648fd4ae0374fd41ba2242d76d10a0931b97a1e56dfd40e5b6d9b50dd8
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1bc933e4

# Article  Sentiment Analysis

if you  have an article to check whether it is positive or negative text

In [159]:
from newspaper import Article
url = input('ENTER THE URL OF THE ARTICLE')
article = Article(url)
article.download()
article.parse()
txt = article.text

print(txt)
txt = steming(txt)
input_data = vectorizer.transform([txt])
prediction = loaded_model.predict(input_data)


if (prediction[0]==0):
  print('This is a Negative Article')
else:
  print('This is a Positive Article')


ENTER THE URL OF THE ARTICLEhttps://sportstar.thehindu.com/wrestling/vinesh-phogat-arrival-in-india-live-updates-new-delhi-airport-paris-olympics-wrestling-disqualification-cas-appeal/article68535363.ece
August 18, 2024 02:30

Felicitation ended at 1:50am

After waiting for almost half a day, Vinesh Phogat’s village got to celebrate her homecoming with her. It’s been a long journey on the road for the Olympian whose journey touched a number of areas in Haryana where she was honoured and consoled. The rally-type journey was organised by Bajrang Punia and Vinesh’s family for the wrestler who garnered immense support as she appealed for a joint silver in the Paris Olympics.

Vinesh began the day in tears, seeing all the support she was getting. Wellwishers and supporters shared glasses of milk, honoured her with garlands and blessings. It was around 1 am by the time she made it to her village Balali. An elaborate felicitation with song and dance awaited her here. She was honoured with a g